# External data preparation for modelling

## Imports

In [1]:
import os, sys, re, glob, subprocess, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader
import time

pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

## Factsheet parsing

In [2]:
ABBREV = {
  'US': 'us',
  'UK': 'gb',
  'Finland': 'fi',
  'Denmark': 'dk',
  'France': 'fr',
  'Spain': 'es',
  'Switzerland': 'ch',
  'Cash': 'cash',
  'Other': 'other',
  'Consumer Staples': 'staples',
  'Consumer Discretionary': 'discretionary',
  'Technology': 'tech',
  'Industrials': 'industrials',
  'Healthcare': 'healthcare',
}

FACTS_COLUMNS = [
  'date', 'aum', 'avg_cap', 'avg_founded', 'holdings',
  'geo_us', 'geo_gb', 'geo_dk', 'geo_es', 'geo_fi', 'geo_ch', 'geo_fr', 'geo_cash',
  'sec_staples', 'sec_tech', 'sec_healthcare', 'sec_discretionary',  'sec_industrials', 'sec_cash',
  'top_1', 'top_2', 'top_3', 'top_4', 'top_5', 'top_6', 'top_7', 'top_8', 'top_9', 'top_10',
  't_acc', 't_inc',
]

companies = pd.read_csv('companies.csv', parse_dates=['first_bought', 'sold_out'])

In [3]:
def pdftotext(filename, flags='-layout'):
  proc = subprocess.Popen(['/usr/bin/pdftotext'] + flags.split() + [filename, '-' ], stdout=subprocess.PIPE)
  out, err = proc.communicate()
  assert proc.wait() == 0, 'pdftotext %s failed' % filename
  return out.decode('utf-8')

def name_to_symbol(name):
  for row in companies[~companies.name_regexp.isnull()].itertuples():
    if re.search(row.name_regexp, name, re.I) is not None:
      return row.symbol
  raise Exception("Can't determine symbol for name: %s" % name)

def get_abbrev(s):
  d = { k.lower().replace(' ', '') : v for (k, v) in ABBREV.items() }
  return d.get(s.lower().replace(' ', '').replace('_', ''), s)

def extract_gbp_b(s):
  m = re.match('£([0-9.]+)bn', s)
  assert m is not None, s
  return float(m.group(1))

def extract_top(filename):
  text = pdftotext(filename)
  it = iter(text.split('\n'))
  res = []
  idx = -1
  for line in it:
    idx = line.find('Top 10 Holdings')
    if idx >= 0: break
  for line in it:
    line = line[max(0, idx-2):min(len(line), idx + 40)]
    line = line.lstrip('• ').strip()
    if line == '': continue
    res.append(line)
    if len(res) >= 10: break
  assert len(res) == 10
  return res

def extract_facts(filename):
  facts = {}
  text = pdftotext(filename, '-f 1 -l 1 -x 0 -y 0 -W 225 -H 780 -layout')

  section = ''
  for line in text.split('\n'):
    line = line.strip()
    
    m = re.match(r'^T Class (Acc|Inc) +[0-9.%]+ +[^ ]+ +([0-9.]+)$', line)
    if m:
      facts['t_%s' % m.group(1).lower()] = float(m.group(2))
      continue

    if 'Geographic Split' in line:
      section = 'geo'
      continue
      
    if 'Sector Split' in line:
      section = 'sec'
      continue

    if line.startswith('As at'):
      line = line[5:].strip()
      if ',' in line: line = line.split(',')[0].strip()
      line = ' '.join(line.split()[:3])

      for fmt in ['%d %b %Y', '%d %B %Y', '%dth %B %Y']:
        try:
          d = datetime.datetime.strptime(line, fmt)
          break
        except:
          pass
      else:
        raise Exception("Couldn't parse date: %s" % line)

      assert 'date' not in facts or facts['date'] == d
      #facts['month'] = d.strftime('%Y-%m')
      facts['date'] = d
      continue
      
    vec = re.split(r'   +', line)
    if len(vec) != 2:
      continue
    vec[1] = vec[1].replace(',', '.')
    vec[1] = vec[1].rstrip('%')
    if vec[0] == 'Fund Size':
      facts['aum'] = extract_gbp_b(vec[1])
    elif vec[0] == 'No. Holdings':
      facts['holdings'] = int(vec[1])
    elif vec[0] == 'Average Co. Founded':
      facts['avg_founded'] = int(vec[1])
    elif vec[0] == 'Average Market Cap':
      facts['avg_cap'] = extract_gbp_b(vec[1])
    elif section != '':
      if section == 'geo' and vec[0] == 'Other' and facts['date'].year == 2016: vec[0] = 'dk'
      facts['%s_%s' % (section, get_abbrev(vec[0]))] = float(vec[1])
      if vec[1] == 'Cash': section = ''

  for i, s in enumerate(extract_top(filename)):
    facts['top_%d' % (i + 1)] = name_to_symbol(s)

  for k in facts.keys():
    assert k in FACTS_COLUMNS, k

  return facts

In [4]:
filenames = glob.glob('/lab/fundsmith/facts/201[6-8]*.pdf')

facts = [ extract_facts(s) for s in filenames ]
facts = pd.DataFrame(facts, columns=FACTS_COLUMNS).fillna(0).sort_values('date').reset_index(drop=True)

#facts

In [5]:
facts.to_csv('facts.csv', index=False)

## Edgar 13F parsing

In [6]:
edgar_urls = {
  '2018-03-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000152002318000005/xslForm13F_X01/Fundsmith_q12018.xml',
  '2017-12-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000152002318000001/xslForm13F_X01/FUNDSMITH_13F_TABLE.xml',
  '2017-09-30': 'https://www.sec.gov/Archives/edgar/data/1569205/000152002317000002/xslForm13F_X01/fundsmith.xml',
  '2017-06-30': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713117004151/xslForm13F_X01/infotable.xml',
  '2017-03-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713117002469/xslForm13F_X01/infotable.xml',
  '2016-12-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713117000731/xslForm13F_X01/infotable.xml',
  '2016-09-30': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713116007934/xslForm13F_X01/infotable.xml',
  '2016-06-30': ['https://www.sec.gov/Archives/edgar/data/1569205/000138713116006514/xslForm13F_X01/infotable.xml',
                 'https://www.sec.gov/Archives/edgar/data/1569205/000138713116006804/xslForm13F_X01/infotable.xml'],
  '2016-03-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713116005089/xslForm13F_X01/infotable.xml',
  '2015-12-31': 'https://www.sec.gov/Archives/edgar/data/1569205/000138713116004227/xslForm13F_X01/infotable.xml',
}

In [7]:
def read_13f(url):
  df = pd.read_html(url)[-1]
  df.columns = [ 'name', 'class', 'cusip', 'usd', 'size', 'sh_prn',  'put_call', 'discretion', 'manager', 'vote_sole', 'vote_shared', 'vote_none' ] 
  return df[3:].reset_index(drop=True).copy()

def process_edgar(date, urls):
  if type(urls) is not list: urls = [urls]

  df = pd.concat([read_13f(u) for u in urls])
  print('%s: %d holdings' % (date, len(df)))
  
  if date == '2017-09-30':
    df.loc[df['name'] == 'PAYPAL HLDGS INC', 'cusip'] = '70450Y103'

  df['date'] = date
  df['shares'] = df['size'].astype('float')
  df['usd'] = pd.to_numeric(df['usd']) * 1000
  df['cusip'] = df.cusip.str.lstrip('0')

  companies = pd.read_csv('companies.csv')
  companies.cusip = companies.cusip.str.lstrip('0')

  df = pd.merge(df, companies[['cusip', 'symbol']], 'left', 'cusip')
  df = df[['date', 'symbol', 'cusip', 'shares', 'usd']]
  if len(df[df.symbol.isnull()]) > 0:
    sys.stderr.write('Could not determine symbol for CUSIPs: %s' % str(df[df.symbol.isnull()]['cusip']))

  return df

In [8]:
edgar = pd.concat([process_edgar(*i) for i in edgar_urls.items()], ignore_index=True)
edgar.date = pd.to_datetime(edgar.date)

2018-03-31: 17 holdings
2017-12-31: 18 holdings
2017-09-30: 18 holdings
2017-06-30: 19 holdings
2017-03-31: 18 holdings
2016-12-31: 18 holdings
2016-09-30: 19 holdings
2016-06-30: 19 holdings
2016-03-31: 17 holdings
2015-12-31: 18 holdings


In [9]:
edgar.to_csv('edgar.csv', index=False)

## Stock prices

In [10]:
companies = pd.read_csv('companies.csv')
symbol_to_currency = companies.set_index('symbol').yahoo_currency.fillna('USD').to_dict()
ezv = pd.read_csv('ezv/ezv.wide.csv', parse_dates=['date'], index_col=['date']).resample('D').ffill()
facts = pd.read_csv('facts.csv', parse_dates=['date'])

In [11]:
def fetch_yahoo(symbol):
  cache = 'yahoo/%s.csv' % symbol
  if not os.path.exists(cache):
    print('Fetching %s from yahoo' % symbol)
    res = pandas_datareader.data.get_data_yahoo(symbol, start='2000-01-01')
    os.makedirs('yahoo', exist_ok=True)
    res.to_csv(cache)
    time.sleep(1)

  return pd.read_csv(cache, index_col=0, parse_dates=[0]).dropna(how='any')

In [12]:
ydata = pd.DataFrame()

for row in companies.itertuples():
  if row.symbol in ['BCR']: continue
  yahoo_symbol = row.yahoo_symbol if type(row.yahoo_symbol) is str else row.symbol
  yahoo_currency = row.yahoo_currency if type(row.yahoo_currency) is str else 'USD'
  
  df = fetch_yahoo(yahoo_symbol)
  df = df.groupby('Date').last()

  for col in df.columns:
    if col == 'Volume':
      conv = df[col]
    else:
      conv = (df[col] * ezv[yahoo_currency] / ezv['USD']).dropna()
    ydata['%s_%s' % (row.symbol, col.replace(' ', ''))] = conv

ydata.index.name = 'Date'
ydata = ydata.reset_index()

ydata = ydata[ydata.Date >= '2016'].reset_index(drop=True)

ydata.to_feather('ydata_usd.feather')

In [13]:
# https://www.londonstockexchange.com/exchange/prices-and-markets/funds/company-summary/UK/25054936.html
df = pd.read_json('t-inc.json')
df = pd.DataFrame({
  'date': df.d.apply(lambda v: pd.Timestamp(v[0] // 1000, unit='s')),
  'GBX': df.d.apply(lambda v: v[1]),
})
df = df.set_index('date')
df['USD'] = (df['GBX'] * ezv['GBX'] / ezv['USD']).dropna()
df.to_csv('t-inc.csv')

## Misc

In [14]:
companies = pd.read_csv('companies.csv')
edgar = pd.read_csv('edgar.csv', parse_dates=['date'])

Companies by sector and geography:

In [15]:
pd.pivot_table(companies, 'symbol', 'sector', 'country', aggfunc=lambda v: ','.join(v), fill_value='')

country,CH,DK,ES,FI,FR,GB,US
sector,,,,,,,
Consumer Discretionary,,,,,,IHG,"DNKN,DPZ,MAR"
Consumer Staples,NESN,,,,OR,"DGE,IMB,RB,ULVR","CL,DPS,EL,PEP,PM,SJM"
Healthcare,,"COLO,NVO",,,,,"BCR,BDX,IDXX,JNJ,SYK,WAT"
Industrials,,,,KNEBV,,ITRK,MMM
Technology,,,AMS,,,SGE,"ADP,FB,INTU,MSFT,ORCL,PG,PYPL,V"
